In [1]:
import scanpy as sc
import scanpy.external as sce
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time
import palantir
import cellrank

Function definitions. One Function per cell

In [2]:
# Time wrapper function
def time_it(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"{func.__name__} executed in {end_time - start_time} seconds")
        return result
    return wrapper

In [3]:
# Cell filtering and QC for UMAP
def filter_cells_for_UMAP(data, min_gen = 500, min_cell = 3, mt_pct = 60, normed = 0, doublet_threshold = 0.24, min_ct = 0, max_genes = 0): #min_ct is removed. Consider restoring it later. You will find it commented out below. Max genes is also removed and commented even further below
    # Working default arguments:
    # data, min_gen = 500, min_cell = 3, mt_pct = 60, normed = 0, doublet_threshold = 0.24, min_ct = 0, max_genes = 0
    adata = data # This is to avoid writing into the file that's entered as an argument
    if min_ct > 0:
        sc.pp.filter_cells(adata, min_counts = min_ct) # Filter cells based on number of RNA reads
    sc.pp.filter_cells(adata, min_genes= min_gen) # Filter cells based on the number of recognized genes
    sc.pp.filter_genes(adata, min_cells = min_cell) # Filter genes based on the minimum number of cells expressing it
    adata_prefilt = adata[adata.obs['doublet_scores'] < doublet_threshold]
    if max_genes > 0:
        adata_prefilt = adata_prefilt[adata_prefilt.obs['n_genes_by_counts'] < max_genes]
        
    if not normed:
        adata_filt = adata_prefilt[adata_prefilt.obs['pct_counts_mt'] < mt_pct] # Filtering based on percentage of mitochondrial genes
    else:
        adata_filt = adata_prefilt
    return adata_filt    

In [4]:
# Processing for UMAP from QC all the way to the end
@time_it # Timing wrapper for benchmarking purposes. Can comment out
def process_for_UMAP(data, normed = 0, leiden_res = 0.8, filtering = 1, min_gen = 500, min_cell = 3, mt_pct = 60, neighbors = 50): # DEFAULT QC VALUES
    # Working default arguments:
    # data, normed = 0, leiden_res = 0.8, filtering = 1, min_gen = 500, min_cell = 3, mt_pct = 60, neighbors = 15
    adata = data # This is to avoid writing into the file that's entered as an argument
    if filtering:
        adata_filt = filter_cells_for_UMAP(data = adata, min_gen = min_gen, min_cell = min_cell, mt_pct = mt_pct) #min ct is removed. Think about restoring it later.
    else:
        adata_filt = adata       
    sc.pp.normalize_total(adata_filt, target_sum=1e4) # Normalize
    sc.pp.log1p(adata_filt) # Log scaling
    sc.pp.highly_variable_genes(adata_filt, min_mean = 0.0125, max_mean = 3, min_disp = 0.5) # Compute differentially expressed genes within the sample
    adata_filt.raw = adata_filt # Store the raw files in its own layer
    #adata_filt = adata_filt[:, adata_filt.var.highly_variable] # Filter on genes that are highly variable
    sc.pp.regress_out(adata_filt, ['total_counts', 'pct_counts_mt']) # Regression. Not sure what that is.
    sc.pp.scale(adata_filt, max_value = 10) # Scale the data
    sc.tl.pca(adata_filt, svd_solver='arpack') # Compute PCA
    sc.tl.tsne(adata_filt) # Calculate tsne
    sc.pp.neighbors(adata_filt, n_neighbors = neighbors) # Calculate neighbors
    sc.tl.leiden(adata_filt, resolution = leiden_res) # Calculate Leiden clusters
    sc.tl.paga(adata_filt) # Calculate PAGA
    sc.pl.paga(adata_filt, plot = 1)  # remove `plot=False` if you want to see the coarse-grained graph
    sc.tl.umap(adata_filt, init_pos='paga') # Plot PAGA
    sc.tl.umap(adata_filt) # Calculate UMAP
    sc.pl.umap(adata_filt, color = ['leiden']) # Plot UMAP and show Leiden clusters
    return adata_filt

In [5]:
# Isolating cells by gene expression. It uses the value as-is so if it's normalized it'll use it
def isolate_cells_by_gene(data, gene, threshold):
    # Now subset_ant_mt_filt contains only the highly variable genes
    data_subset = data[data[:, gene].X > threshold]
    return data_subset

In [6]:
# This function filters the leiden clusters that are positive for a specified gene
# diff is boolean specifying if differential expresion is already done
# Default assumes you didn't do your ranked gene expression analysis
# Default method is kept as the default method for the function
# threshold is the threshold of expression
def filter_clusters_by_gene(data, gene, threshold = 0.5, diff = 0, method = None):
    # Load your AnnData object
    adata = data
    if diff == 0:
        sc.tl.rank_genes_groups(adata, groupby = 'leiden_samp', method = method)
    # Extract the DataFrame for the differential expression results
    de_results = pd.DataFrame(adata.uns['rank_genes_groups']['names'])
    # Define a threshold for significant expression (adjust as needed)
    expression_threshold = threshold
    # Find clusters with significant gene expression
    significant_clusters = []
    for cluster in de_results.columns:
        gene_presence = de_results[cluster].str.contains(gene)
        gene_expression = adata.uns['rank_genes_groups']['logfoldchanges'][cluster][gene_presence]
        if any(gene_expression >= expression_threshold):
            significant_clusters.append(cluster)
    # Subset the data to include only cells from the significant clusters
    adata_subset = adata[adata.obs['leiden_samp'].isin(significant_clusters)].copy()
    return adata_subset

In [7]:
# This function makes a density scatter plot like what one might see with flow analysis.
# The X and Y axes defaults are specified to my specific use, but they can be changed
def scatter_density_plot(data, background = 'grey', cmap = 'hsv', upxlim = None, loxlim = None, upylim = None, loylim = None, size = None, bw_adjust = 0.5, x = 'Gastric stemness', y = 'Intestinal stemness'):
    # Extract the data from the AnnData object
    x_data = data.obs[x].values
    y_data = data.obs[y].values
    
    # Creating the density plot
    plt.figure()
    
    # Set the axes
    plt.xlim(loxlim, upxlim);
    plt.ylim(loylim, upylim);
    
    # Create the base scatter plot
    plt.scatter(x_data, y_data, c = 'gray', s = size, edgecolor='none')

    # Create the heatmap on top    
    sns.kdeplot(x = x_data, y = y_data, cmap = "hsv", shade=True, bw_adjust = bw_adjust)
    
    # Label the axes
    plt.xlabel(x)
    plt.ylabel(y)
    
    # Show the plot
    plt.show()
    
    return(plt)

In [8]:
# Function for mapping names to columns
# Must create the mapping set and feed it here
def map_to_column(data, map_set, column = 'Localization'):
    data.obs[column + '_old'] = data.obs[column]
    data.obs[column] = data.obs[column].map(map_set)
    print(data.obs[column])
    print(data.obs[column + '_old'])
    return 'Mapping function done'

Setting up the environmental and misc variables alongside starting the timer to measure script execution time

In [9]:
# Execution start begin with timing
start_time = time.time()
sc.settings.verbosity = 3
sc.set_figure_params(dpi = 150)
global_res = 0.5

In [10]:
path = 'S:/data cache/code_in_out/agr2/agr2_init_files/Aline/raw_data/'
antrum = sc.read_h5ad(path + 'agr2_unfilt_antrum.h5ad')

In [12]:
filt = filter_cells_for_UMAP(antrum)

filtered out 2331 cells that have less than 500 genes expressed
filtered out 14348 genes that are detected in less than 3 cells


In [14]:
sc.pp.pca(filt)

computing PCA
    with n_comps=50
    finished (0:00:06)


C:\anaconda_envs\spyder\Lib\site-packages\scanpy\preprocessing\_pca.py:229: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata.obsm['X_pca'] = X_pca


In [15]:
sce.pp.harmony_integrate(filt, key = 'Patient')

2024-08-29 01:26:29,050 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...


AttributeError: 'NoneType' object has no attribute 'split'